In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp
import math
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("../class")
from Flow_class import Flow
from Reaction_class import Reaction
from Transport_class import Transport
from Concentrations import Concentrations

In [3]:
Nx=100
Ny=1
Lx=0.045
Ly=0.01
#Ly=1.0
Nt=10000
hx=Lx/Nx
hy=Ly/Ny
phys_dims=[Lx,Ly]
g=pp.CartGrid([Nx,Ny],phys_dims)
g.compute_geometry()
#pp.plot_grid(g,figsize=(15,12))

In [4]:
# Permeability
perm = pp.SecondOrderTensor(1e-9*np.ones(g.num_cells))                     
#f_lambda= lambda x,y,z: math.sin(4*math.pi*x)
f_lambda=lambda x,y,z:0.0

tol=1e-5
bc_lambda=lambda x,y,z:0.3*int(x<tol)

# Collect all parameters in a dictionary
parameters = {"second_order_tensor": perm, "f_lambda": f_lambda, "bc_lambda": bc_lambda}

In [5]:
flow=Flow(g,parameters,"MVEM")
flow.set_bc()
flow.set_source()
p=flow.solve()

MVEM


/home/dell/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [6]:
flux=flow.get_flux(p)

In [7]:
print(flux)

[6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.66666667e-11 6.66666667e-11 6.66666667e-11
 6.66666667e-11 6.666666

In [8]:
psi1=np.zeros(g.num_cells)
psi2=np.zeros(g.num_cells)
psi3=np.zeros(g.num_cells)
psi4=np.zeros(g.num_cells)
psi5=np.zeros(g.num_cells)
psi6=np.zeros(g.num_cells)
rd=np.zeros(g.num_cells)

In [9]:
unity = np.ones(g.num_cells)
porosity=0.3*unity
aperture=1

In [10]:
reaction_data={"kd":1.83e-7,
               "K_eq":4.45e-7,
               "K_sol":10e9,
               "porosity":porosity }

In [11]:
reaction=Reaction(g,reaction_data)

In [12]:
K_eq=reaction.data[pp.PARAMETERS]["reaction"]["K_eq"]

In [13]:
bc_type_Ca=["dir","dir"]
bc_value_Ca=[0.0,0.0]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_Ca=lambda x,y,z:0.0

bc_type_CaSiO3=["dir","dir"]
bc_value_CaSiO3=[0.0,0.0]
#init_cond_CaSiO3=lambda x,y,z:int(x<0.5)
init_cond_CaSiO3=lambda x,y,z:1.0

bc_type_CO2=["dir","dir"]
bc_value_CO2=[0.3691108,0.0]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_CO2=lambda x,y,z:1.0e-3

bc_type_SiO2=["dir","dir"]
bc_value_SiO2=[0.0,0.0]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_SiO2=lambda x,y,z:0.0

bc_type_H_piu=["dir","dir"]
bc_value_H_piu=[1.0e-5,0.0]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_H_piu=lambda x,y,z:1.0e-7



bc_HCO3_in=K_eq*bc_value_CO2[0]/bc_value_H_piu[0]
print("bc_HCO3_in=")
print(bc_HCO3_in)
#bc_HCO3_out=K_eq*bc_value_CO2[1]/bc_value_H_piu[1]
bc_HCO3_out=0.0

bc_type_HCO3=["dir","dir"]
bc_value_HCO3=[bc_HCO3_in,bc_HCO3_out]
#init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_HCO3=lambda x,y,z:0.0

bc_HCO3_in=
0.0164254306


In [14]:
data_6reagents = {"bc_type_Ca": bc_type_Ca,
            "bc_value_Ca": bc_value_Ca,
            "bc_type_CO2": bc_type_CO2,
            "bc_value_CO2": bc_value_CO2,
            "bc_type_HCO3": bc_type_HCO3,
            "bc_value_HCO3": bc_value_HCO3,
            "bc_type_CaSiO3": bc_type_CaSiO3,
            "bc_value_CaSiO3": bc_value_CaSiO3,
            "bc_type_SiO2": bc_type_SiO2,
            "bc_value_SiO2": bc_value_SiO2,
            "bc_type_H_piu": bc_type_H_piu,
            "bc_value_H_piu": bc_value_H_piu,
            "time_step": 34.56,
            "t_max": 345600,
            "method": "Implicit",
            "lambda_lin_decay":0,
            "init_cond_Ca":init_cond_Ca,
            "init_cond_CO2":init_cond_CO2,
            "init_cond_CaSiO3":init_cond_CaSiO3,
            "init_cond_SiO2":init_cond_SiO2,
            "init_cond_HCO3":init_cond_HCO3,
            "init_cond_H_piu":init_cond_H_piu,}

In [15]:
concentrations=Concentrations(g,Nt,data_6reagents)
concentrations.set_initial_cond(K_eq)

In [16]:
psi1,psi2,psi3,psi4,psi5=concentrations.compute_psi(0,psi1,psi2,psi3,psi4,psi5)

In [17]:
bc_psi1,bc_psi2,bc_psi3,bc_psi4,bc_psi5=concentrations.set_bc_psi()

In [18]:
data_reagent_psi1= {
            "bc_type": bc_type_Ca,
            "bc_value": bc_psi1,
            "mass_weight": porosity * aperture,
            "darcy_flux":flux,
            "lambda_lin_decay":0.,
            "method": "Implicit",
            "time_step": 34.56,
            "t_max": 345600,
}

data_reagent_psi2= {
            "bc_type": bc_type_H_piu,
            "bc_value": bc_psi2,
            "mass_weight": porosity * aperture,
            "darcy_flux":flux,
            "lambda_lin_decay":0.,
            "method": "Implicit",
            "time_step": 34.56,
            "t_max": 345600,
}

data_reagent_psi3= {
            "bc_type": bc_type_CO2,
            "bc_value": bc_psi3,
            "mass_weight": porosity * aperture,
            "darcy_flux":flux,
            "lambda_lin_decay":0.,
            "method": "Implicit",
            "time_step": 34.56,
            "t_max": 345600,
}

data_reagent_psi4= {
            "bc_type": bc_type_CaSiO3,
            "bc_value": bc_psi4,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.zeros(g.num_faces),
            "lambda_lin_decay":0.,
            "method": "Implicit",
            "time_step": 34.56,
            "t_max": 345600,
}

data_reagent_psi5= {
            "bc_type": bc_type_SiO2,
            "bc_value": bc_psi5,
            "mass_weight": porosity * aperture,
            "darcy_flux":flux,
            "lambda_lin_decay":0.,
            "method": "Implicit",
            "time_step": 34.56,
            "t_max": 345600,
}

In [19]:
psi1_transport=Transport(g,data_reagent_psi1)
#psi1=psi1_transport.set_bc(psi1)
psi1_transport.set_bc()
psi1_lhs,psi1_rhs_b,psi1_rhs_matrix=psi1_transport.get_transport_lhs_rhs()

psi2_transport=Transport(g,data_reagent_psi2)
#psi2=psi2_transport.set_bc(psi2)
psi2_transport.set_bc()
psi2_lhs,psi2_rhs_b,psi2_rhs_matrix=psi2_transport.get_transport_lhs_rhs()

psi3_transport=Transport(g,data_reagent_psi3)
#psi3=psi3_transport.set_bc(psi3)
psi3_transport.set_bc()
psi3_lhs,psi3_rhs_b,psi3_rhs_matrix=psi3_transport.get_transport_lhs_rhs()

psi4_transport=Transport(g,data_reagent_psi4)
#psi4=psi4_transport.set_bc(psi4)
psi4_transport.set_bc()
psi4_lhs,psi4_rhs_b,psi4_rhs_matrix=psi4_transport.get_transport_lhs_rhs()

psi5_transport=Transport(g,data_reagent_psi5)
#psi5=psi5_transport.set_bc(psi5)
psi5_transport.set_bc()
psi5_lhs,psi5_rhs_b,psi5_rhs_matrix=psi5_transport.get_transport_lhs_rhs()

In [20]:
n_steps = int(np.round(concentrations.data[pp.PARAMETERS]["6reagents"]["t_max"] / concentrations.data[pp.PARAMETERS]["6reagents"]["time_step"]))

In [21]:
dt=psi1_transport.data[pp.PARAMETERS]["transport"]["time_step"]

In [22]:
save_every=100
exporter = pp.Exporter(g, file_name="6_reagents",folder_name="solution_6_reagents_2D")

In [23]:
print("Ca:")
print(concentrations.Ca[:,0])
print("CaSiO3:")
print(concentrations.CaSiO3[:,0])
print("H_piu:")
print(concentrations.H_piu[:,0])
print("CO2:")
print(concentrations.CO2[:,0])

Ca:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
CaSiO3:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
H_piu:
[1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07
 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07
 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07
 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07
 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07 1.e-07
 1.e-07 1.e-07

In [24]:
for i in range(1,n_steps+1,1):
#for i in range(1,500,1):
    if np.isclose((i-1) % save_every, 0):
        exporter.write_vtu({"Ca":concentrations.Ca[:,i-1], "CaSiO3":concentrations.CaSiO3[:,i-1], "H_piu":concentrations.H_piu[:,i-1], "HCO3":concentrations.HCO3[:,i-1], "CO2":concentrations.CO2[:,i-1],"SiO2":concentrations.SiO2[:,i-1]}, time_step=int((i-1) // save_every))

    psi1,psi2,psi3,psi4,psi5=concentrations.compute_psi(i-1,psi1,psi2,psi3,psi4,psi5)
    rd=reaction.compute_rd_6_reagents(concentrations.Ca[:,i-1],concentrations.SiO2[:,i-1],concentrations.H_piu[:,i-1],concentrations.CaSiO3[:,i-1],rd)
    psi1,psi2,psi3,psi4,psi5=concentrations.one_step_transport_reaction(psi1,psi2,psi3,psi4,psi5,
                                               psi1_lhs,psi1_rhs_b,psi1_rhs_matrix,
                                               psi2_lhs,psi2_rhs_b,psi2_rhs_matrix,
                                               psi3_lhs,psi3_rhs_b,psi3_rhs_matrix,
                                               psi4_lhs,psi4_rhs_b,psi4_rhs_matrix,
                                               psi5_lhs,psi5_rhs_b,psi5_rhs_matrix,
                                               rd)
    
    concentrations.compute_concentration(psi1,psi2,psi3,psi4,psi5,i,K_eq)



exporter.write_vtu({"Ca":concentrations.Ca[:,-1], "CaSiO3":concentrations.CaSiO3[:,-1], "H_piu":concentrations.H_piu[:,-1], "HCO3":concentrations.HCO3[:,-1], "CO2":concentrations.CO2[:,-1],"SiO2":concentrations.SiO2[:,-1]}, time_step=(n_steps // save_every))
time_steps = np.arange(
    0, psi1_transport.data[pp.PARAMETERS]["transport"]["t_max"] + dt, save_every * dt
)
exporter.write_pvd(time_steps)
    

/home/dell/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


KeyboardInterrupt: 

In [ ]:
print("Ca=")
print(concentrations.Ca[:,-1])
print("CaSiO3=")
print(concentrations.CaSiO3[:,-1])
print("CO2=")
print(concentrations.CO2[:,-1])
print("HCO3=")
print(concentrations.HCO3[:,-1])
print("SiO2=")
print(concentrations.SiO2[:,-1])
print("H_piu=")
print(concentrations.H_piu[:,-1])

In [ ]:
exporter.write_pvd(time_steps)